In [1]:
import numpy as np

In [2]:
polarity_donald = np.load("Datasets/Validation/US/hashtag_donaldtrump_polarity.npy")
subjectivity_donald = np.load("Datasets/Validation/US/hashtag_donaldtrump_subjectivity.npy")
bnb_donald = np.load("Datasets/Validation/US/hashtag_donaldtrump_sentiment_BNB.npy")

In [3]:
polarity_biden = np.load("Datasets/Validation/US/hashtag_joebiden_polarity.npy")
subjectivity_biden = np.load("Datasets/Validation/US/hashtag_joebiden_subjectivity.npy")
bnb_biden = np.load("Datasets/Validation/US/hashtag_joebiden_sentiment_BNB.npy")

In [4]:
import pandas as pd

In [5]:
df_donald = pd.read_csv("Datasets/Validation/US/hashtag_donaldtrump.csv",lineterminator='\n')
df_donald["vader"] = polarity_donald
df_donald["model"] = bnb_donald
df_donald["opinion"] = subjectivity_donald

In [6]:
df_biden = pd.read_csv("Datasets/Validation/US/hashtag_joebiden.csv",lineterminator='\n')
df_biden["vader"] = polarity_biden
df_biden["model"] = bnb_biden
df_biden["opinion"] = subjectivity_biden

In [7]:
print(list(df_donald.keys()))
print(df_donald["country"])

import time

['created_at', 'tweet_id', 'tweet', 'likes', 'retweet_count', 'source', 'user_id', 'user_name', 'user_screen_name', 'user_description', 'user_join_date', 'user_followers_count', 'user_location', 'lat', 'long', 'city', 'country', 'continent', 'state', 'state_code', 'collected_at', 'vader', 'model', 'opinion']
0         United States of America
1                              NaN
2         United States of America
3                              NaN
4         United States of America
                    ...           
970914                         NaN
970915                         NaN
970916                         NaN
970917                         NaN
970918                         NaN
Name: country, Length: 970919, dtype: object


In [8]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New York', 'New Mexico', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
stateCodes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
stateMapping = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 
                  'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NY': 'New York', 'NM': 'New Mexico', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT':  'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV':  'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'}

In [23]:
def filter(df_donald):

    df_donald['country']=df_donald['country'].replace({"United States of America" : "US", "United States" : "US"})
    df_donald = df_donald.loc[df_donald['country'] == "US"] 
    df_filtered = df_donald
    print(len(df_donald.index))
    print(len(df_filtered.index))
    print(df_donald.iterrows())
    i = 0
    
    # loop = tqdm(len(df_donald.index),leave=True)
    for index, row in df_donald.iterrows():
        i+=1
        flag = 0
        t1 = time.time()
        if row.state in states:
            flag = 2
            t2 = time.time()
            print(t2-t1)
            print("state")
        elif row.state_code in stateCodes:
            row['state'] = stateMapping[row.state_code]
            flag = 1
            t2 = time.time()
            print(t2-t1)
            print("state code")
        elif (isinstance(row.user_location,float)==False):
            words = row.user_location.split(',')
            for word in words:
                check = word.strip()
                if check in states:
                    row['state'] = check
                    flag = 1
                    break
                elif check in stateCodes:
                    row['state'] = stateMapping[check]
                    flag = 1
                    break
            t2 = time.time()
            print(t2-t1)
            print("location")
        if (flag==1):
            df_filtered.loc[index, 'state'] = row['state']
            print(row['state'])
            t2 = time.time()
            print(t2-t1)
            print("update state")
        elif (flag == 0):
            df_filtered = df_filtered.drop(index = index)
            print(row.country)
            t2 = time.time()
            print(t2-t1)
            print("drop")
        # loop.update(1)
        print(i)

In [9]:

df_donald = pd.read_csv("Datasets/Validation/US/hashtag_donaldtrump_filtered.csv",lineterminator='\n')
df_biden = pd.read_csv("Datasets/Validation/US/hashtag_joebiden_filtered.csv",lineterminator='\n')

In [54]:
tweets_location_plan1 = pd.DataFrame(index = [state for state in states],columns=
        ['Trump Positive count',
        'Trump Negative count',
        'Trump Neutral count',
        'Trump Total count',
        'Biden Positive count',
        'Biden Negative count',
        'Biden Neutral count',
        'Biden Total count',
        
        'Trump Positive Sum',
        'Trump Negative Sum',
        'Trump Total Sum',
        'Biden Positive Sum',
        'Biden Neutral count',
        'Biden Total Sum'])

tweets_location_plan2 = pd.DataFrame(index = [state for state in states],columns=
        ['Trump Positive count',
        'Trump Negative count',
        'Trump Neutral count',
        'Trump Total count',
        'Biden Positive count',
        'Biden Negative count',
        'Biden Neutral count',
        'Biden Total count'])

tweets_location_plan3 = pd.DataFrame(index = [state for state in states],columns=
        ['Trump Positive count',
        'Trump Negative count',
        'Trump Neutral count',
        'Trump Total count',
        'Biden Positive count',
        'Biden Negative count',
        'Biden Neutral count',
        'Biden Total count',
        
        'Trump Positive Sum',
        'Trump Negative Sum',
        'Trump Total Sum',
        'Biden Positive Sum',
        'Biden Neutral count',
        'Biden Total Sum'])

tweets_location_plan4 = pd.DataFrame(index = [state for state in states],columns=
        ['Trump Positive count',
        'Trump Negative count',
        'Trump Neutral count',
        'Trump Total count',
        'Biden Positive count',
        'Biden Negative count',
        'Biden Neutral count',
        'Biden Total count',
        
        'Trump Positive Sum',
        'Trump Negative Sum',
        'Trump Total Sum',
        'Biden Positive Sum',
        'Biden Neutral count',
        'Biden Total Sum'])


tweets_location_plan1 = tweets_location_plan1.fillna(0)
tweets_location_plan2= tweets_location_plan2.fillna(0)
tweets_location_plan3 = tweets_location_plan3.fillna(0)
tweets_location_plan4 = tweets_location_plan4.fillna(0)


In [65]:
def update_df(df_donald,name,vader_df,trained_df,op_vader,op_trained):
    
    if (name == "Trump"):
        for index, row in df_donald.iterrows():
            # update vader count (!Idea1)
            start = time.time()
            if (row.vader>0):

                vader_df.at[row.state, 'Trump Positive Sum'] += row.vader
                vader_df.at[row.state,'Trump Total Sum'] += row.vader
                vader_df.at[row.state,'Trump Positive count'] += 1
                vader_df.at[row.state,'Trump Total count'] += 1
                end = time.time()
                print(end-start)
                print("update vader+")
            elif (row.vader<0):
                vader_df.at[row.state,'Trump Negative Sum'] += row.vader
                vader_df.at[row.state,'Trump Total Sum'] += row.vader
                vader_df.at[row.state,'Trump Negative count'] += 1
                vader_df.at[row.state,'Trump Total count'] += 1
                end = time.time()
                print(end-start)
                print("update vader-")
            else:
                vader_df.at[row.state,'Trump Neutral count'] += 1
                end = time.time()
                print(end-start)
                print("update vader=")

            start = time.time()
            # update trained count (!Idea2)
            if (row.model>0):
                
                trained_df.at[row.state,'Trump Positive count'] += 1
                trained_df.at[row.state,'Trump Total count'] += 1
                end = time.time()
                print(end-start)
                print("update trained+")
                
            else:
                trained_df.at[row.state,'Trump Negative count'] += 1
                trained_df.at[row.state,'Trump Total count'] += 1
                end = time.time()
                print(end-start)
                print("update trained-")

            start = time.time()
            # update weighted count (!Idea3)
            if (row.vader>0):
                op_vader.at[row.state,'Trump Positive Sum'] += row.vader*row.opinion
                op_vader.at[row.state,'Trump Total Sum'] += row.vader*row.opinion
                op_vader.at[row.state,'Trump Positive count'] += 1
                op_vader.at[row.state,'Trump Total count'] += 1
                end = time.time()
                print(end-start)
                print("vader weight+")
            elif (row.vader<0):
                op_vader.at[row.state,'Trump Negative Sum'] += row.vader*row.opinion
                op_vader.at[row.state,'Trump Total Sum'] += row.vader*row.opinion
                op_vader.at[row.state,'Trump Negative count'] += 1
                op_vader.at[row.state,'Trump Total count'] += 1
                end = time.time()
                print(end-start)
                print("vader weight-")
            else:
                op_vader.at[row.state,'Trump Neutral count'] += 1
                end = time.time()
                print(end-start)
                print("vader weight=")

            start = time.time()
            # update weighted count (!Idea4)
            if (row.model>0):
                op_trained.at[row.state,'Trump Positive Sum'] += row.vader*row.opinion
                op_trained.at[row.state,'Trump Total Sum'] += row.vader*row.opinion
                op_trained.at[row.state,'Trump Positive count'] += 1
                op_trained.at[row.state,'Trump Total count'] += 1
                end = time.time()
                print(end-start)
                print("trained weight+")
            else:
                op_trained.at[row.state,'Trump Negative Sum'] += row.vader*row.opinion
                op_trained.at[row.state,'Trump Total Sum'] += row.vader*row.opinion
                op_trained.at[row.state,'Trump Negative count'] += 1
                op_trained.at[row.state,'Trump Total count'] += 1
                end = time.time()
                print(end-start)
                print("trained weight-")
    
    elif (name == "Biden"):
        for index, row in df_donald.iterrows():
            # update vader count (!Idea1)
            if (row.vader>0):
                vader_df.at[row.state, 'Biden Positive Sum'] += row.vader
                vader_df.at[row.state,'Biden Total Sum'] += row.vader
                vader_df.at[row.state,'Biden Positive count'] += 1
                vader_df.at[row.state,'Biden Total count'] += 1
            elif (row.vader<0):
                vader_df.at[row.state,'Biden Negative Sum'] += row.vader
                vader_df.at[row.state,'Biden Total Sum'] += row.vader
                vader_df.at[row.state,'Biden Negative count'] += 1
                vader_df.at[row.state,'Biden Total count'] += 1
            else:
                vader_df.at[row.state,'Biden Neutral count'] += 1
            

            # update trained count (!Idea2)
            if (row.model>0):
                trained_df.at[row.state,'Biden Positive count'] += 1
                trained_df.at[row.state,'Biden Total count'] += 1
                
            else:
                trained_df.at[row.state,'Biden Negative count'] += 1
                trained_df.at[row.state,'Biden Total count'] += 1

            # update weighted count (!Idea3)
            if (row.vader>0):
                op_vader.at[row.state,'Biden Positive Sum'] += row.vader*row.opinion
                op_vader.at[row.state,'Biden Total Sum'] += row.vader*row.opinion
                op_vader.at[row.state,'Biden Positive count'] += 1
                op_vader.at[row.state,'Biden Total count'] += 1
            elif (row.vader<0):
                op_vader.at[row.state,'Biden Negative Sum'] += row.vader*row.opinion
                op_vader.at[row.state,'Biden Total Sum'] += row.vader*row.opinion
                op_vader.at[row.state,'Biden Negative count'] += 1
                op_vader.at[row.state,'Biden Total count'] += 1
            else:
                op_vader.at[row.state,'Biden Neutral count'] += 1

            # update weighted count (!Idea4)
            if (row.model>0):
                op_trained.at[row.state,'Biden Positive Sum'] += row.vader*row.opinion
                op_trained.at[row.state,'Biden Total Sum'] += row.vader*row.opinion
                op_trained.at[row.state,'Biden Positive count'] += 1
                op_trained.at[row.state,'Biden Total count'] += 1
            else:
                op_trained.at[row.state,'Biden Negative Sum'] += row.vader*row.opinion
                op_trained.at[row.state,'Biden Total Sum'] += row.vader*row.opinion
                op_trained.at[row.state,'Biden Negative count'] += 1
                op_trained.at[row.state,'Biden Total count'] += 1
    
    return (vader_df,trained_df,op_vader,op_trained)

In [66]:
tweets_location_plan1, tweets_location_plan2, tweets_location_plan3, tweets_location_plan4 =update_df(df_donald,"Trump",tweets_location_plan1, tweets_location_plan2, tweets_location_plan3, tweets_location_plan4)

0.0013632774353027344
update vader=
0.00012636184692382812
update trained-
0.00043702125549316406
vader weight=
0.0009250640869140625
trained weight-
0.0008089542388916016
update vader+
4.076957702636719e-05
update trained-
0.0008029937744140625
vader weight+
0.0008802413940429688
trained weight-
0.0008180141448974609
update vader-
4.1961669921875e-05
update trained-
0.0008115768432617188
vader weight-
0.0007970333099365234
trained weight-
0.0007960796356201172
update vader+
3.790855407714844e-05
update trained-
0.0007951259613037109
vader weight+
0.0008249282836914062
trained weight-
0.000885009765625
update vader+
8.106231689453125e-05
update trained+
0.0009572505950927734
vader weight+
0.0008060932159423828
trained weight+
0.0008058547973632812
update vader-
3.790855407714844e-05
update trained-
0.0007779598236083984
vader weight-
0.0009579658508300781
trained weight-
0.0008327960968017578
update vader+
4.482269287109375e-05
update trained+
0.0008199214935302734
vader weight+
0.0008

KeyboardInterrupt: 

In [ ]:
tweets_location_plan1, tweets_location_plan2, tweets_location_plan3, tweets_location_plan4 =update_df(df_donald,"Trump",tweets_location_plan1, tweets_location_plan2, tweets_location_plan3, tweets_location_plan4)

In [ ]:
tweets_location_plan1.to_csv("Datasets/Validation/US/statewise_total_idea1.csv")
tweets_location_plan2.to_csv("Datasets/Validation/US/statewise_total_idea2.csv")
tweets_location_plan3.to_csv("Datasets/Validation/US/statewise_total_idea3.csv")
tweets_location_plan4.to_csv("Datasets/Validation/US/statewise_total_idea4.csv")